# Build a language model for a specific subject


One of the state-of-the art solutions to build NLP application is using word embeddings to
compute similarities between texts. Generally, they are vector representations of words that
are capable of capturing the context of a word in a document or relation with other words.

Word2vec is a two-layer neural network that processes text. Its input is a text corpus and its output is a set of vectors: feature vectors for words in that corpus. The purpose and usefulness of Word2vec is to group the vectors of similar words together in vectorspace. This way. it is able to detect similarities mathematically [1].

A pre-trained set of vectors containing part of Google News dataset can be downloaded and used [2].
In case of technical fields, however, in order to achieve a better accuracy, a new language model should be trained,
in order to obtain the specific vocabulary that may be missing from the news dataset.

## Build up an input corpus

The first step is to build a corpus of specialized words, that can be used to get the vocabulary.
In order to do that, we decided to use papers stored in Zenodo [3], since it offers a free end point for
querying. In order to do that, we raise multiple queries to the server and we store the results locally.
The approach that we selected was to download the pdf files and transform them to txt, using a command lines tool
for Linux. To speed up the process and to avoid duplicates, we save the files in the '/tmp' folder and check, each time, if the file is already there. In case it is, we load the text, which is already pre-processed. I case of builing a language model, it is better to not have duplicates, since it can change the values attached to words.

The next step is to cleanup the text. In order to to that, the first step is to remove all punctation and replace non-alpha characters to spaces. Them all the words are transformed to lowercase and lemmatized to their basis word. Then,
all the stop words are removed, together with the non-English and short words. In this case, it is important to keep the  abbreviations, technical words which are not the English dictionary and short words related to the concerned field.
Them all the abreviations are replaced by the corresponding words sequence, to be able to compute similarities right. The clean-up texts are then saved in the corresponding text files in the '/tmp' folder.

## Build a language model

The corpus of documents is directly sent to word2vec library of gensim [4] and then saved in a binary file for further
use.


[1] https://skymind.ai/wiki/word2vec
[2] https://code.google.com/archive/p/word2vec/
[3] https://zenodo.org/
[4] https://radimrehurek.com/gensim/models/word2vec.html

In [55]:
%run "NLP_clustering.ipynb"
%run "Utils_Zenodo.ipynb"


def get_entries_from_zenodo(query):
    
    total_articles = get_zenodo_entries(query)
    documents = []
    
    for article in total_articles:
        doi = article['doi']
        mod_doi = doi.replace('/', '-')

        #look in tmp folder if there is a file containing the doi
        # if there is, just read the file and move to the next entry
        if os.path.isfile('/tmp/' + doi + ".txt"):
            with open('/tmp/' + doi + ".txt") as f:
                doc_list = f.read().split(' ')
        else:
            print(article['files'][0]['links']['self'])
            # download the file and preprocess in the information
            doc_list = save_pdf_and_get_text(article['files'][0]['links']['self'])
            # overwrite the doi temporary file
            with open('/tmp/' + mod_doi + ".txt", 'w') as f:
                f.write(' '.join(doc_list))
                f.close()
            # add the text to the corpus   
            if doc_list:
                documents.append(doc_list)
 
    return documents


if __name__ == "__main__":
    #documents is a list of lists
    documents = get_entries_from_zenodo(SEARCH_QUERY)
    documents.extend(get_entries_from_zenodo("solar irradiance"))
    documents.extend(get_entries_from_zenodo("photovoltaic"))
    documents.extend(get_entries_from_zenodo("Renewable Energy"))
    documents.extend(get_entries_from_zenodo("solar pond"))
    documents.extend(get_entries_from_zenodo("solar observations"))
    documents.extend(get_entries_from_zenodo("global horizontal irradiance"))
    documents.extend(get_entries_from_zenodo("irradiance"))
    documents.extend(get_entries_from_zenodo("solar collector"))
    documents.extend(get_entries_from_zenodo("downwelling longwave"))
    documents.extend(get_entries_from_zenodo("clear sky"))
    documents.extend(get_entries_from_zenodo("diffuse radiation"))
    documents.extend(get_entries_from_zenodo("water vapor"))
    documents.extend(get_entries_from_zenodo("snowdepth"))
    documents.extend(get_entries_from_zenodo("bi-directional reflectance"))
    documents.extend(get_entries_from_zenodo("snowcover"))
    documents.extend(get_entries_from_zenodo("airmass"))
    documents.extend(get_entries_from_zenodo("downwelling"))
    
    model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
    model.train(documents,total_examples=len(documents),epochs=10)
    model.wv.save_word2vec_format(MODEL_TO_TRAIN, binary=True)
    

[]
['10.5281-zenodo.3362794', '10.5281-zenodo.290171', '10.5281-zenodo.2553228', '10.5281-zenodo.2564916', '10.5281-zenodo.1412072', '10.5281-zenodo.1092032', '10.5281-zenodo.1089217', '10.5281-zenodo.2540160', '10.5281-zenodo.1414115', '10.5281-zenodo.3362722', '10.5281-zenodo.1123991', '10.5281-zenodo.1232319', '10.5281-zenodo.1060980', '10.5281-zenodo.1133921', '10.1109-PIMRC.2018.8580891', '10.5281-zenodo.1091100', '10.5281-zenodo.1123953', '10.5281-zenodo.1059537', '10.5281-zenodo.1083239', '10.5281-zenodo.1453758', '10.5281-zenodo.1061549', '10.5281-zenodo.1099100', '10.5281-zenodo.1092467', '10.5281-zenodo.1419902', '10.5281-zenodo.1471164', '10.5281-zenodo.1466328', '10.5281-zenodo.160192', '10.5281-zenodo.2553783', '10.5281-zenodo.53751', '10.5281-zenodo.49306', '10.5281-zenodo.1078617', '10.5281-zenodo.848993', '10.5281-zenodo.1080814', '10.5281-zenodo.1108430', '10.5281-zenodo.1490513', '10.5281-zenodo.2598431', '10.9734-BJAST-2017-33408', '10.5281-zenodo.1162482', '10.5281-

already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
['10.5281-zenodo.3362794', '10.5281-zenodo.290171', '10.5281-zenodo.2553228', '10.5281-zenodo.2564916', '10.5281-zenodo.1412072', '10.5281-zenodo.1092032', '10.5281-zenodo.1089217', '10.5281-zenodo.2540160', '10.5281-zenodo.1414115', '10.5281-zenodo.3362722', '10.5281-zenodo.1123991', '10.5281-zenodo.1232319', '10.5281-zenodo.1060980', '10.5281-zenodo.1133921', '10.1109-PIMRC.2018.8580891', '10.5281-zenodo.1091100', '10.5281-zenodo.1123953', '10.5281-zenodo.1059537', '10.5281-zenodo.1083239', '10.5281-zenodo.1453758', '10.5281-zenodo.1061549', '10.5281-zenodo.1099100', '10.5281-zenodo.1092467', '10.5281-zenodo.1419902', '10.5281-zenodo.1471164', '10.5281-zenodo.1466328', 

already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already 

already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already in list
already 

already in list
already in list
https://zenodo.org/api/files/f85d01f9-d0b3-4dc5-8b34-31c79f718fca/1523346545_Volume%205%20Issue%204.pdf
['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
already in list
https://zenodo.org/api/files/a717a525-bfc2-4010-859b-0a130beeede4/2118mechatroj01.pdf
['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
https://zenodo.org/api/files/a3c85c23-ff4b-4937-9b77-8e77963a3c45/77.pdf
['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
already in list
already in list
https://zenodo.org/api/files/d264e21d-38e5-4c67-b24c-f3f358943a40/Onipede2142017BJAST32785.pdf
['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmas

...

['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
https://zenodo.org/api/files/0a8f0567-93a3-46ce-8b66-aa405db012e0/Formation%20of%20Tungsten%20Oxide%20Nanowires%20by%20Electron-Beam-Enhanced%20Oxidation%20of%20WS2%20Nanotubes%20and%20Platelets.pdf
['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
https://zenodo.org/api/files/2d2deafe-1e5f-469c-8521-cb415115c443/Estimation%20of%20the%20effect%20of%20temperature%2C%20the%20concentration%20of%20oxygen%20and%20catalysts%20on%20the%20oxidation%20of%20the%20thermoanthracite%20carbon%20material.pdf
['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
https://zenodo.org/api/files/eb4ec808-3198-40ac-ac39-62956483dc1d/10009094.pdf
['irradiation', 'photo

['10.5281-zenodo.3362794', '10.5281-zenodo.290171', '10.5281-zenodo.2553228', '10.5281-zenodo.2564916', '10.5281-zenodo.1412072', '10.5281-zenodo.1092032', '10.5281-zenodo.1089217', '10.5281-zenodo.2540160', '10.5281-zenodo.1414115', '10.5281-zenodo.3362722', '10.5281-zenodo.1123991', '10.5281-zenodo.1232319', '10.5281-zenodo.1060980', '10.5281-zenodo.1133921', '10.1109-PIMRC.2018.8580891', '10.5281-zenodo.1091100', '10.5281-zenodo.1123953', '10.5281-zenodo.1059537', '10.5281-zenodo.1083239', '10.5281-zenodo.1453758', '10.5281-zenodo.1061549', '10.5281-zenodo.1099100', '10.5281-zenodo.1092467', '10.5281-zenodo.1419902', '10.5281-zenodo.1471164', '10.5281-zenodo.1466328', '10.5281-zenodo.160192', '10.5281-zenodo.2553783', '10.5281-zenodo.53751', '10.5281-zenodo.49306', '10.5281-zenodo.1078617', '10.5281-zenodo.848993', '10.5281-zenodo.1080814', '10.5281-zenodo.1108430', '10.5281-zenodo.1490513', '10.5281-zenodo.2598431', '10.9734-BJAST-2017-33408', '10.5281-zenodo.1162482', '10.5281-zen

['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
https://zenodo.org/api/files/0dbb09bd-9f03-41b1-beda-4876263cf08c/1395.pdf
['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
https://zenodo.org/api/files/0d860165-6d3f-4f67-b836-c7fdc1e2e122/Khudyakova_Enviromis%202018_DOI.pdf
['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
['10.5281-zenodo.3362794', '10.5281-zenodo.290171', '10.5281-zenodo.2553228', '10.5281-zenodo.2564916', '10.5281-zenodo.1412072', '10.5281-zenodo.1092032', '10.5281-zenodo.1089217', '10.5281-zenodo.2540160', '10.5281-zenodo.1414115', '10.5281-zenodo.3362722', '10.5281-zenodo.1123991', '10.5281-zenodo.1232319', '10.5281-zenodo.1060980', '10.5281-zenodo.1133921', '10.1109-

['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
https://zenodo.org/api/files/52d31225-8440-4ccd-bdd1-9e945ff998f6/Sullivan%20templates%20Lyon%202017.pdf
['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
https://zenodo.org/api/files/5593d0b2-855e-496b-a187-641cd335e38c/poster_Svensson_etal.pdf
['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
already in list
https://zenodo.org/api/files/00ad1c00-20d1-4c99-8fdd-2f7af581e929/thesis_zairamberdinas.pdf
['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
['10.5281-zenodo.3362794', '10.5281-zenodo.290171', '10.5281-zenodo.2553228', '10.5281-zenodo.25

['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
already in list
already in list
https://zenodo.org/api/files/57b658bf-54fc-4e14-b862-741302e865ce/article.pdf
['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
https://zenodo.org/api/files/67d2750e-85bc-4297-9b5f-a1afb956d57b/fulltext.pdf
['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
already in list
https://zenodo.org/api/files/15775bcd-9d18-4ecf-bee9-223d42d6e77d/Peralta-Maraver_et_al_2018.pdf
['irradiation', 'photovoltaic', 'irradiance', 'snowdepth', 'snowcover', 'reflectance', 'bi-directional', 'airmass', 'longwave', 'downwelling', 'sky']
https://zenodo.org/api/files/381780c8-5630-402f-bf70-119b30a56ff5/fulltext.pdf
['irradiation', 'ph

In [56]:
print(len(model.wv.vocab))
#print(documents)

#print(model.wv.most_similar('solar'))
#print(model.wv.most_similar('photovoltaic'))

print(model.wv.most_similar('irradiance'))
print(model.wv.most_similar('solar'))
print(model.wv.most_similar('photovoltaic'))

model.wv.most_similar("sky")



15669


[('cloudless', 0.7406683564186096),
 ('overcast', 0.7078748941421509),
 ('longwave', 0.6878471374511719),
 ('airmass', 0.680314302444458),
 ('noon', 0.6768925786018372),
 ('sunset', 0.6757547855377197),
 ('equator', 0.6633356213569641),
 ('cloudy', 0.6620073914527893),
 ('zenith', 0.6586511135101318),
 ('incidence', 0.6575011014938354)]

In [34]:
print(len(model.wv.vocab))
print(documents)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

